# Find hospitals closest to an incident

The `network` module of the ArcGIS API for Python can be used to solve different types of network analysis operations. In this sample, we see how to find the hospital that is closest to an incident.  

## Closest facility

The closest facility solver provides functionality for finding out the closest locations to a particular input point. This solver would be useful in cases when you have an incident and need to find the closest facility or need to get information on the travel time and the distance to each of the facilities from an incident point for reporting purposes.

![](http://desktop.arcgis.com/en/arcmap/latest/extensions/network-analyst/GUID-96C273DB-6A24-4D42-AADA-975A33B44F3D-web.png)

When finding closest facilities, you can specify how many to find and whether the direction of travel is toward or away from them. The closest facility solver displays the best routes between incidents and facilities, reports their travel costs, and returns driving directions.

## Table of Contents

* [Connect to your GIS](#Connect-to-your-GIS)
* [Create a Network Layer](#Create-a-Network-Layer)
* [Create a Facilities Layer](#Create-a-Facilities-layer)
* [Create Incidents Layer](#Create-Incidents-Layer)
* [Solve for Closest Facility](#Solve-for-Closest-Facility)
* [Analyze Results in a Table](#Analyze-Results-in-a-Table)
* [Conclusion](#Conclusion)

### Connect to your GIS
As a first step, import the necessary libraries and establish a connection to your organization, which could be an ArcGIS Online organization or an ArcGIS Enterprise portal.

In [3]:
import traceback

import pandas as pd
from IPython.display import HTML

from arcgis.gis import GIS
from arcgis.network import ClosestFacilityLayer
from arcgis import geocoding
from arcgis.features import Feature, FeatureSet

# Connect to your GIS
gis = GIS(url_to_gis, username, password)

### Create a Network Layer
To perform any network analysis (such as finding the closest facility, the best route between multiple stops, or service area around a facility), you would need to create a `NetworkLayer` object. In this sample, since we are solving for closest facilities, we need to create a `ClosestFacilityLayer` which is a type of `NetworkLayer`.

To create any `NetworkLayer` object, you would need to provide the URL to the appropriate network analysis service. Hence, in this sample, we provide a `ClosestFacility` URL to create a `ClosestFacilityLayer` object. 

Since all ArcGIS Online organizations already have access to those routing services, you can access this URL through the `GIS` object's `helperServices` property. If you have your own ArcGIS Server-based map service with network analysis capability enabled, provide the URL for this service.

Let's start by checking to make sure we have our portal configured for Network Analysis, and getting the ClosestFacility URL from our `GIS`:

In [4]:
try:
    analysis_url = gis.properties.helperServices.closestFacility.url
    print("Closest Facility URL successfully retrieved")
except Exception as e:
    traceback.print_exc()
    raise Exception(f"Network Analysis is not properly configured "\
                    f"on your Portal: {e}")

Closest Facility URL successfully retrieved


Create a `ClosestFacilityLayer` object from the `arcgis.network` module using the retrieved URL:

In [5]:
cf_layer = ClosestFacilityLayer(analysis_url, gis = gis)

### Create a Facilities layer
In this sample, we will be looking for the closest hospital (facility) to an incident location. Even though we are interested in finding out the closest one, it would still be helpful to get the information on the distance and travel time to all of them for reference purposes.

In the code below, we need to geocode the hospitals' addresses as well as do the reverse geocode for the incident location which has been supplied in the latitude/longitude format.

To perform the geocode operations, we will use the `geocoding` module of the ArcGIS API.

In this sample, we geocode addresses of hospitals to create the facility layer. In your workflows, this could be any layer of facilities to route from an incident. In this scenario, we'll create a list of hospitals in the Coachella Valley region of southern California.

In [6]:
hospitals_addresses = [
    'San Gorgonio Memorial Hospital,600 N. Highland Springs Ave.,'\
        'Banning,CA,92220',
    'Desert Regional Medical Center,1150 N. Indian Canyon Dr.,'\
        'Palm Springs,CA,92262',
    'JFK Memorial Hospital,47111 Monroe St.,Indio,CA,92201',
    'Hi-Desert Medical Center,6601 White Feather Rd.,'\
        'Joshua Tree,CA,92252',
    'Eisenhower Health,39000 Bob Hope Dr.,Rancho Mirage,CA,92270']

Loop through each address and geocode it. The geocode operation returns a list of matches for each address. We pick the first result and extract the coordinates from it and construct a `Feature` object out of it. Then we combine all the `Feature`s representing the hospitals into a `FeatureSet` object.

In [7]:
hosp_feat_list = []

for address in hospitals_addresses:
    hit = geocoding.geocode(address)[0]
    hosp_feat = Feature(geometry=hit['location'],
                        attributes=hit['attributes'])
    hosp_feat_list.append(hosp_feat)

Construct a `FeatureSet` using each hospital `Feature`.

In [8]:
hospitals_fset = FeatureSet(features=hosp_feat_list, 
                            geometry_type='esriGeometryPoint', 
                            spatial_reference={'latestWkid': 4326})

Let's draw our hospitals on a map

In [10]:
gis = GIS("https://idt.esri.com/portal", "jyaist_ds19")
map1 = gis.map('Palm Springs, CA')
map1

Enter password: ········


MapView(layout=Layout(height='400px', width='100%'))

In [11]:
map1.zoom = 8

In [12]:
map1.draw(hospitals_fset,
          symbol={"type": "esriSMS","style": "esriSMSSquare",
                  "color": [76,115,0,255],"size": 8,})

### Create Incidents Layer
Similarly, let us create the incident layer

In [13]:
incident_coords = '-116.391287,33.808053'
reverse_geocode = geocoding.reverse_geocode(
    {"x": incident_coords.split(',')[0], 
     "y": incident_coords.split(',')[1]})

incident_feature = Feature(geometry=reverse_geocode['location'], 
                           attributes=reverse_geocode['address'])

In [14]:
incident_fset = FeatureSet([incident_feature],
                           geometry_type='esriGeometryPoint',
                           spatial_reference={'latestWkid': 4326})

Add the incident to the map

In [15]:
map1.draw(incident_fset,
          symbol={"type": "esriSMS","style": "esriSMSDiamond",
                  "size": 12, "color":[255,0,0,255]})

### Solve for Closest Facility
By default the closest facility service would return only the closest location, so we need to specify explicitly the `default_target_facility_count` parameter as well as `return_facilities` to get multiple results.

In [16]:
result = cf_layer.solve_closest_facility(
    incidents=incident_fset,
    facilities=hospitals_fset,
    default_target_facility_count=5,
    return_facilities=True,
    impedance_attribute_name='TravelTime',
    accumulate_attribute_names=['Kilometers','TravelTime'])

Inspect the result dictionary

In [17]:
result.keys()

dict_keys(['messages', 'routes', 'facilities'])

Let's use the `routes` dictionary to construct line features out of the routes to display on the map. From the `routes` dictionary will find out the proper key to get each route: the `features` key.

In [18]:
result['routes'].keys()

dict_keys(['fieldAliases', 'geometryType', 'spatialReference', 'features'])

In [19]:
result['routes']['features'][0].keys()

dict_keys(['attributes', 'geometry'])

Construct line features out of the routes that are returned.

In [20]:
line_feat_list = []
for line_dict in result['routes']['features']:
    f1 = Feature(line_dict['geometry'], line_dict['attributes'])
    line_feat_list.append(f1)

In [21]:
routes_fset = FeatureSet(line_feat_list, 
    geometry_type=result['routes']['geometryType'],
    spatial_reference= result['routes']['spatialReference'])

Add the routes back to the map. The route to the closest hospital is in red

In [22]:
map1.draw(routes_fset)

### Analyze Results in a Table
Since we parsed the routes as a `FeatureSet`, we can display the attributes easily as a `pandas` `DataFrame`. We'll sort the dataframe according to the `Name` attribute of the `solve_closest_facility` task results.

In [23]:
df1 = routes_fset.sdf.sort_values(by=['Name'])
df1

,FacilityCurbApproach,FacilityID,FacilityRank,IncidentCurbApproach,IncidentID,Name,ObjectID,SHAPE,Shape_Length,Total_Kilometers,Total_Miles,Total_TravelTime
3,1,1,4,2,1,I-10 W - Location 1,4,"{""paths"": [[[-116.39135814899998, 33.807921482...",0.597683,56.731844,35.251534,34.454366
1,1,2,2,2,1,I-10 W - Location 2,2,"{""paths"": [[[-116.39135814899998, 33.807921482...",0.180008,17.331207,10.769113,16.149298
2,1,3,3,2,1,I-10 W - Location 3,3,"{""paths"": [[[-116.39135814899998, 33.807921482...",0.253096,24.967126,15.513853,19.268972
4,1,4,5,2,1,I-10 W - Location 4,5,"{""paths"": [[[-116.39135814899998, 33.807921482...",0.719424,70.715838,43.940785,49.228632
0,1,5,1,2,1,I-10 W - Location 5,1,"{""paths"": [[[-116.39135814899998, 33.807921482...",0.090572,9.728557,6.045045,9.620388


Add the hospital addresses and incident address to this table and display only the relevant columns

In [24]:
df1['facility_address'] = hospitals_addresses
df1['incident_address'] = [incident_feature.attributes['Match_addr'] \
                          for i in range(len(hospitals_addresses))]

In [25]:
pd.set_option('max_colwidth', 80)

df1[['facility_address','incident_address','Total_Miles',
     'Total_TravelTime']].sort_values(by=['Total_TravelTime'])


,facility_address,incident_address,Total_Miles,Total_TravelTime
0,"Eisenhower Health,39000 Bob Hope Dr.,Rancho Mirage,CA,92270","I-10 W, Thousand Palms, California, 92276",6.045045,9.620388
1,"Desert Regional Medical Center,1150 N. Indian Canyon Dr.,Palm Springs,CA,92262","I-10 W, Thousand Palms, California, 92276",10.769113,16.149298
2,"JFK Memorial Hospital,47111 Monroe St.,Indio,CA,92201","I-10 W, Thousand Palms, California, 92276",15.513853,19.268972
3,"San Gorgonio Memorial Hospital,600 N. Highland Springs Ave.,Banning,CA,92220","I-10 W, Thousand Palms, California, 92276",35.251534,34.454366
4,"Hi-Desert Medical Center,6601 White Feather Rd.,Joshua Tree,CA,92252","I-10 W, Thousand Palms, California, 92276",43.940785,49.228632


### Conclusion
Using the `network` module of the ArcGIS API for Python, you can solve for closest facilities from an incident location.